<a href="https://colab.research.google.com/github/shu65/coffee-tuning/blob/main/coffee_tuning_blog%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna gspread pandas

     |████████████████████████████████| 286kB 5.6MB/s 
     |████████████████████████████████| 1.1MB 36.5MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 481kB 40.6MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 133kB 36.7MB/s 
     |████████████████████████████████| 112kB 48.0MB/s 
  Created wheel for alembic: filename=alembic-1.5.3-py2.py3-none-any.whl size=155547 sha256=53d56f46f2644f778edfbb4bd0dc8e9875f0ec971423efb904b95c8aa3f22cf3
  Stored in directory: /root/.cache/pip/wheels/7f/ed/1b/a66a0cbca75fd3e374bd5cc60c443e5675f10ef1f1f78ec31d
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha256=dd6eb3f8f0e2dc10068d6a79ecccea98d295c300381fd322c890e44eab932488
  Stored in directory: /root/.cache/pip/wheels/ad/10/d3/aeb26e20d19045e2a68e5d3cbb57432e11b5d9c92c99f98d47
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sha

In [2]:
# Google スプレッドシートの承認

from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [3]:
# スプレッドシートからデータを取ってくる
import pandas as pd

ss_name = "shu65コーヒーデータ"
workbook = gc.open(ss_name)
worksheet = workbook.get_worksheet(1)
df = pd.DataFrame(worksheet.get_all_records())
df = df.set_index('淹れた日')
df

,豆,豆を買った店,豆を買った日,豆の量(g),ミルの時間 (sec),トータルの時間 (sec),蒸らし時間 (sec),スコア,コメント
淹れた日,,,,,,,,,
2020/12/29,マンデリン,店0,2020/12/28,9,5,240,30,5,マンデリンのいい感じの苦みが強調されていて久々に入れたにしてはよくできた印象
2020/12/30,マンデリン,店0,2020/12/28,9,5,180,30,4,マンデリンのいい感じの苦みが強調されていて久々に入れたにしてはよくできた印象


In [4]:

# 探索空間の定義

import optuna

search_space={
    "豆の量(g)": optuna.distributions.IntUniformDistribution(8, 12),
    "ミルの時間 (sec)": optuna.distributions.IntUniformDistribution(3, 15),
    "トータルの時間 (sec)": optuna.distributions.IntUniformDistribution(180, 300),
    "蒸らし時間 (sec)": optuna.distributions.IntUniformDistribution(20, 40),
    }
score_column = 'スコア'

In [5]:
# 現在までのデータをstudyに登録

import optuna

sampler = optuna.samplers.TPESampler(multivariate=True)
study = optuna.create_study(direction='maximize', sampler=sampler)

for record_i, record in df.iterrows():
  print(record.to_dict())
  params = {}
  for key in search_space.keys():
    params[key] = record[key]
  trial = optuna.trial.create_trial(
    params=params,
    distributions=search_space,
    value=record[score_column])
  study.add_trial(trial)

/usr/local/lib/python3.6/dist-packages/optuna/samplers/_tpe/sampler.py:191: ExperimentalWarning:

``multivariate`` option is an experimental feature. The interface can change in the future.

[I 2021-01-30 02:32:09,287] A new study created in memory with name: no-name-9477651e-3950-413f-abc8-5385514f3198


{'豆': 'マンデリン', '豆を買った店': '店0', '豆を買った日': '2020/12/28', '豆の量(g)': 9, 'ミルの時間 (sec)': 5, 'トータルの時間 (sec)': 240, '蒸らし時間 (sec)': 30, 'スコア': 5, 'コメント': 'マンデリンのいい感じの苦みが強調されていて久々に入れたにしてはよくできた印象'}
{'豆': 'マンデリン', '豆を買った店': '店0', '豆を買った日': '2020/12/28', '豆の量(g)': 9, 'ミルの時間 (sec)': 5, 'トータルの時間 (sec)': 180, '蒸らし時間 (sec)': 30, 'スコア': 4, 'コメント': 'マンデリンのいい感じの苦みが強調されていて久々に入れたにしてはよくできた印象'}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: ExperimentalWarning:

create_trial is experimental (supported from v2.0.0). The interface can change in the future.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: ExperimentalWarning:

add_trial is experimental (supported from v2.0.0). The interface can change in the future.



In [6]:
# 次のパラメータの出力
trial = study._ask()

new_params = {}
for key, space in search_space.items():
  new_params[key] = trial._suggest(key, space)

for key in  ["豆の量(g)", "ミルの時間 (sec)",  "トータルの時間 (sec)",  "蒸らし時間 (sec)"]:
  print(key, new_params[key])

豆の量(g) 9
ミルの時間 (sec) 13
トータルの時間 (sec) 199
蒸らし時間 (sec) 33
